In [1]:
import math
import numpy as np
from numba import cuda, float32, int32

***
# FIR filter
***
<font size="4">
The finite impulse response (FIR) filter is given by equation \eqref{fir}

\begin{equation}
    y_i = \sum_{j=0}^{N-1} h_j x_{i+\left\lceil \frac{N}{2} \right\rceil+1-j} 
    \label{fir} \tag{1}
\end{equation}

where *x* is an input signal, *h* is the impulse response of the filter and *y* is an output signal.
Filtration is realised by convolving the signal with impulse response of the filter.

$x_i$ is equal 0 for $i < 0$
    
***    
</font>

Examples:

In [2]:
np.convolve([0, 1, 2, 3, 4], [0, 1, 2], mode='same')

array([ 0,  1,  4,  7, 10])

Examples:
    
```

x = [0, 1, 2, 3, 4]
h = [0, 1, 2]

```

```
 x = [0, 1, 2, 3, 4]      y = 
     [1, 0]               [0 ]
     [2, 1, 0]            [1 ]
        [2, 1, 0]         [4 ]
           [2, 1, 0]      [7 ]
              [2, 1]      [10]
```

```
y[0] = h[0]*x[1]+h[1]*x[0]           = 0
y[1] = h[0]*x[2]+h[1]*x[1]*h[2]*x[0] = 1
y[2] = h[0]*x[3]+h[1]*x[2]*h[2]*x[1] = 4
y[3] = h[0]*x[4]+h[1]*x[3]+h[2]*x[2] = 7
y[4] = h[1]*x[4]+h[2]*x[3] =           10
```

### CPU implementation

In [3]:
def convolve_cpu(x, h):
    x = np.asarray(x)
    h = np.asarray(h)
    M = len(x)
    N = len(h)
    y = np.zeros(M, dtype=x.dtype)
    offset = math.ceil(N/2)-1
        
    for i in range(M):
        # This loop can be parallelized.
        value = 0.0
        for j in range(N):
            k = i + offset - j    
            if k >= 0 and k < M:
                value += x[k]*h[j]
        y[i] = value
        
    return y


convolve_cpu([0, 1, 2, 3, 4], [0, 1, 2])

array([ 0,  1,  4,  7, 10])

In [4]:
def test_convolve(func):
    # Test simple case
    x = np.array([0, 1, 2, 3, 4], dtype=np.float32)
    h = np.array([0, 1, 2], dtype=np.float32)
    result = func(x, h)
    np.testing.assert_equal(result, [0, 1, 4, 7, 10])
    
    # Test a bit longer filter.
    x = np.arange(10).astype(np.float32)
    h = np.arange(5).astype(np.float32)
    result = func(x, h)
    np.testing.assert_equal(result, np.convolve(x, h, mode='same'))
        
    # Test if it gives the "same" results as nump.convolve. 
    rng = np.random.default_rng(29062021)
    for i in range(100):
        intervals = rng.integers(low=1, high=30, size=2)
        h_len, x_len = sorted(intervals)
        x = rng.random(x_len).astype(np.float32)
        h = rng.random(h_len).astype(np.float32)
        result = func(x, h)
        np.testing.assert_almost_equal(result, np.convolve(x, h, mode='same'), decimal=5)
    
    print("All tests passed.")
    
    
def benchmark_convolve(func, n=20, x_size=10000, h_size=256):
    import time

    times = []
    for i in range(n):
        x = np.random.rand(x_size).astype(np.float32)
        h = np.random.rand(h_size).astype(np.float32)
        start = time.time()
        result = func(x, h)
        end = time.time()
        times.append(end-start)
    print("Benchmark result: ")
    print(f"Average processing time: " 
        + f"{np.mean(times):.4f} "
        + f"seconds (+/- {np.std(times).item():.4f}), "
        + f"median: {np.median(times):.4f}")
    
test_convolve(convolve_cpu)
benchmark_convolve(convolve_cpu, n=2)

All tests passed.
Benchmark result: 
Average processing time: 1.2039 seconds (+/- 0.0141), median: 1.2039


### Naive CUDA kernel implementation

In [5]:
@cuda.jit
def convolve_gpu_kernel(y, x, h):
    i = cuda.grid(1)
    M = len(x)
    N = len(h)
    offset = int32(math.ceil(N/2)-1)
    
    if i >= len(y):
        return
    
    value = float32(0.0)
    
    for j in range(N):
        k = i + offset - j
        if k >= 0 and k < M:
            value += x[k]*h[j]
    
    y[i] = value
    
def convolve_gpu(y, x, h):
    if y is None:
        y = cuda.device_array(x.shape, dtype=x.dtype)
    block_size = (256, )
    grid_size = (math.ceil(len(y)/block_size[0]), )
    convolve_gpu_kernel[grid_size, block_size](y, x, h)
    return y.copy_to_host()

In [6]:
test_convolve(lambda x, h: convolve_gpu(None, x, h))
benchmark_convolve(lambda x, h: convolve_gpu(None, x, h))

All tests passed.
Benchmark result: 
Average processing time: 0.0010 seconds (+/- 0.0000), median: 0.0010


### Profiling the kernel

In [7]:
%%writefile 2_1_convolve_global_memory.py

import math
import numpy as np
from numba import cuda, float32, int32
from tests import benchmark_convolve

@cuda.jit
def convolve_gpu_kernel(y, x, h):
    i = cuda.grid(1)
    M = len(x)
    N = len(h)
    offset = int32(math.ceil(N/2)-1)
    
    if i >= len(y):
        return
    
    value = float32(0.0)
    
    for j in range(N):
        k = i + offset - j
        if k >= 0 and k < M:
            value += x[k]*h[j]
    
    y[i] = value
    
def convolve_gpu(y, x, h):
    if y is None:
        y = cuda.device_array(x.shape, dtype=x.dtype)
    block_size = (256, )
    grid_size = (math.ceil(len(y)/block_size[0]), )
    convolve_gpu_kernel[grid_size, block_size](y, x, h)
    return y.copy_to_host()

benchmark_convolve(lambda x, h: convolve_gpu(None, x, h))

Overwriting 2_1_convolve_global_memory.py


In [8]:
! nvprof python 2_1_convolve_global_memory.py

==20258== NVPROF is profiling process 20258, command: python 2_1_convolve_global_memory.py
Benchmark result: 
Average processing time: 0.0293 seconds (+/- 0.0539), median: 0.0229
==20258== Profiling application: python 2_1_convolve_global_memory.py
==20258== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   80.85%  1.78268s       100  17.827ms  17.257ms  28.191ms  cudapy::__main__::convolve_gpu_kernel$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
                   12.27%  270.61ms       300  902.03us  1.1200us  2.8563ms  [CUDA memcpy DtoH]
                    6.88%  151.75ms       200  758.76us     896ns  2.2119ms  [CUDA memcpy HtoD]
      API calls:   86.48%  2.08104s       300  6.9368ms  13.580us  29.745ms  cuMemcpyDtoH
                    5.73%  138.00ms       200  690.01us  7.8420us  2.0586ms  cuMemcpyHtoD
                    4.95